In [2]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb

# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
    if '3_feat' in feat or 'tfidf' in feat:
        continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
def simple_ens(model_name,k=3,rnd=233,lr=0.05,feature_fraction=0.9,bagging_fraction=0.9,
               bag_frec=3,met='binary_logloss',max_d=3):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    cls_auc_res = [0,0,0,0,0,0]
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_auc_l,all_val_auc_l = 0,0
    
    for i in range(6):
        val_loss_l,train_loss_l = 0,0
        val_auc_l,train_auc_l = 0,0
        fold_cnt = 0
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test = test_x

            # share params
            params = {
                    'application': 'binary',
                    #'num_leaves': 8,
                    #'lambda_l1': 1,
                    'lambda_l2': 1.0,
                    'max_depth': max_d,
                    'metric': met, # or auc
                    'data_random_seed': 2,
                    'learning_rate':lr,
                    # 'bagging_fraction': bagging_fraction,
                    # 'bagging_freq':bag_frec,
                    'feature_fraction': feature_fraction,

                    }
            if met == 'auc':
                s_round = 60
            else:
                s_round = 30
            # train for each class
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            try:
                train_pred = model.predict(curr_x)
                tmp_test_pred = model.predict(hold_out_x)
                
                curr_train_loss = log_loss(curr_y[:,i],train_pred)
                curr_val_loss = log_loss(hold_out_y[:,i],tmp_test_pred)
                
                curr_train_auc = roc_auc_score(curr_y[:,i],train_pred)
                curr_val_auc = roc_auc_score(hold_out_y[:,i],tmp_test_pred)
                
                print('ls',curr_train_loss,curr_val_loss,'auc',curr_train_auc,curr_val_auc)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
                val_auc_l += curr_val_auc
                train_auc_l += curr_train_auc
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_auc_l = train_auc_l/k
        val_auc_l = val_auc_l/k
        print(list_classes[i], lr, feature_fraction, max_d)
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class auc train',train_auc_l,'auc val',val_auc_l)
        cls_auc_res[i] = val_auc_l
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/6
        all_val_loss_l += val_loss_l/6
        all_train_auc_l += train_auc_l/6
        all_val_auc_l += val_auc_l/6
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all auc avg',all_train_auc_l,all_val_auc_l)
    print('=======================================================')
    return test_pred, cls_auc_res

print('done')

file path ../features/fasttext_cnn2d_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v2_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cudnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_gru_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_lstm_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn2d_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v2_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cudnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_gru_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_lstm_v1_5_feat.pkl
(159571, 6) (15

In [3]:
lgb_res,tmp_auc_res = simple_ens('lgb',10,666,0.05,0.6)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = lgb_res
sample_submission.to_csv("../results/lgb_log_csv_fold10_stratified.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# add 7 base fasttext NN models
# all loss avg 0.0319116484218 0.0358161833583 all auc avg 0.994546891137 0.991249510282 PUB 9866

# rm tfidf test
# all loss avg 0.0319555637279 0.0357756335619 all auc avg 0.994512718368 0.991251471241

# change to stratified
# all loss avg 0.0316412744167 0.0357232681944 all auc avg 0.994584962017 0.991293307537 pub 9866

# change to base model 5 fold, add word batch, tilli, lgb feat
# feat dim 217
# all loss avg 0.031983129767200906 0.035387924581 all auc avg 0.9945427088311004 0.99188514127 PUB 9870

# add more base models
# feat dim 247, all loss avg 0.031885221777487156 0.0353958  all auc avg 0.9945986685511962 0.9919487331094685

# change early stopping to 30, and test later part
# all loss avg 0.03208696729295824 0.035406264613808025 all auc avg 0.9945261773637045 0.991946174432887

# add fasttext lstm v1
# all loss avg 0.031977558955160815 0.03537909655655411 all auc avg 0.9945299073418443 0.9919307178575658

# add muse base model, feat dim 295, lower loss, but lower auc
# all loss avg 0.03196610276261484 0.03530962013098028 all auc avg 0.9945635742334386 0.9918682952070021

0 fold: ls 0.07082459202841758 0.07788605946484535 auc 0.9903426793823865 0.9871873590023756
1 fold: ls 0.06982598330960191 0.07139385538410228 auc 0.990686271422562 0.989887401222387
2 fold: ls 0.06790106411808906 0.07565829379748586 auc 0.991320182628064 0.9884964511634059
3 fold: ls 0.06983979218949386 0.07339745823793384 auc 0.9906588669020915 0.9889679834599027
4 fold: ls 0.06741270193086214 0.07812503238333891 auc 0.9914377720980595 0.9878862189881131
5 fold: ls 0.06807489243581936 0.07668913324607354 auc 0.9913034267747648 0.9875417104630684
6 fold: ls 0.0688250142078539 0.07813235678803032 auc 0.9910154667707481 0.9874609776100283
7 fold: ls 0.07193139366832126 0.07072960517919263 auc 0.9899497335563221 0.9901281492811761
8 fold: ls 0.06672564410504497 0.0739717223905832 auc 0.9916703648527927 0.989441759131684
9 fold: ls 0.06778541274279691 0.07780505430228057 auc 0.9912947133249533 0.9876472439696968
toxic 0.05 0.6 3
this class avg train 0.06891464907363011 avg val 0.07537885

In [4]:
# find best params for each column, early stopping = 30

best_pred = np.zeros((153164,6))
val_auc_res = [0,0,0,0,0,0]
for lr in [0.095,0.075,0.05]:
    for max_d in [3,4]:
        for s_rate in [0.4,0.5,0.6]:
            print('learning rate',lr,'max depth',max_d,'feature fraction',s_rate)
            lgb_res,tmp_auc_res = simple_ens('lgb',k=10,rnd=666,lr=lr,
                                 feature_fraction=s_rate,bagging_fraction=0.9,
                                 bag_frec=3,met='binary_logloss',max_d=max_d)
            # check for each cls
            for i in range(6):
                # find better params for this class
                if tmp_auc_res[i] > val_auc_res[i]:
                    val_auc_res[i] = tmp_auc_res[i]
                    best_pred[:,i] = lgb_res[:,i]
                    print('FIND BETTER PARAMS',lr,max_d,s_rate,list_classes[i])
            print('TEST PARAM DONE ------------------------------------------')

print(val_auc_res)
print(np.mean(val_auc_res))
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = best_pred
sample_submission.to_csv("../results/lgb_grid_search_fold10_stratified.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')
                
            
# best auc params
# toxic 0.05,4,0.5
# severe toxic 0.075 3 0.5
# obs 0.075 3 0.6
# threat 0.095 3 0.5
# insult 0.075 0.5 4
# hate 0.05 0.5 3

# TEST PARAM DONE ------------------------------------------
# [0.9884873039634368, 0.9920148292218363, 0.9954573112511824, 
#  0.994769628570376, 0.9898761527824232, 0.9912880359235366]
# 0.9919822102854652 PUB 9870

learning rate 0.095 max depth 3 feature fraction 0.4
0 fold: ls 0.07032462456130593 0.07805699435609477 auc 0.9905121746503449 0.9870612878734342
1 fold: ls 0.06974897663322285 0.07138105109791182 auc 0.9906984488470143 0.9899317956551441
2 fold: ls 0.06815247974063551 0.07585130986072906 auc 0.9912312411011892 0.9884968588673801
3 fold: ls 0.06864901949572397 0.0730807203164039 auc 0.9910716888088433 0.9891643608741898
4 fold: ls 0.06727486101688862 0.07794403552131811 auc 0.9915146906352263 0.9878843151252116
5 fold: ls 0.06704811250215989 0.07621279001221803 auc 0.9916172994110399 0.9877310088315667
6 fold: ls 0.06929262102619256 0.07806146016691114 auc 0.9908240006973146 0.9874999161393723
7 fold: ls 0.06995425345665494 0.07066819701767738 auc 0.9906007045766996 0.990189530449026
8 fold: ls 0.06724442290862763 0.07405331140650659 auc 0.9915008233395682 0.9894236711804492
9 fold: ls 0.06669899870358421 0.07776443032831963 auc 0.9916431737366077 0.987679249216744
toxic 0.095 0.4 3
th

toxic 0.095 0.5 3
this class avg train 0.06829849594898602 avg val 0.07531853820262238
this class auc train 0.9911424027394478 auc val 0.9884608456905746
0 fold: ls 0.017842469680742313 0.021186128634271842 auc 0.9942820118060772 0.9911127990884923
1 fold: ls 0.018097047918148472 0.020467768860946287 auc 0.9941101190180066 0.9918363875174073
2 fold: ls 0.017929866923086842 0.021078370672597376 auc 0.9942357259092655 0.9912841024180276
3 fold: ls 0.016913751360810816 0.019734735555816205 auc 0.9951219365154628 0.9926814311938219
4 fold: ls 0.018107165914831837 0.021094702022251197 auc 0.9941034678863316 0.9910985567793393
5 fold: ls 0.017462436858332694 0.019882285176737846 auc 0.9946358165689045 0.992271133755487
6 fold: ls 0.01822093995974973 0.019126004205842095 auc 0.9940124272708062 0.9929884784269603
7 fold: ls 0.01765011661513987 0.020214662092221416 auc 0.9944722031416281 0.9920751611543152
8 fold: ls 0.017601558304689537 0.020983408587054095 auc 0.9945110533014986 0.99124425742

0 fold: ls 0.03581681017570127 0.03722214042156464 auc 0.996131063250916 0.9957113610015595
1 fold: ls 0.03544066955716216 0.0368681308233893 auc 0.9962031988174622 0.9955860721029781
2 fold: ls 0.034362875880456945 0.03898989419179695 auc 0.9964254415336992 0.9954020630182214
3 fold: ls 0.035632861539899065 0.033742809850886415 auc 0.9961600647382544 0.9965342014340264
4 fold: ls 0.03524006625201157 0.04213699024641663 auc 0.9962659416268901 0.9945094771661536
5 fold: ls 0.03222712711150447 0.03759990237522869 auc 0.9969063894758798 0.9955722322633999
6 fold: ls 0.03585500576261543 0.03760058301718717 auc 0.996124129928869 0.9952542123348818
7 fold: ls 0.03535871901453186 0.03842921659266206 auc 0.9962144975454159 0.9955120896125498
8 fold: ls 0.034077808183623826 0.03885094447785947 auc 0.9965091045808782 0.9951542878264383
9 fold: ls 0.0351975065174004 0.03852646900785779 auc 0.9962578669114952 0.9953594519530633
obscene 0.095 0.6 3
this class avg train 0.0349209449994907 avg val 0.

1 fold: ls 0.003974395158865864 0.006295550305220559 auc 0.9993185903419417 0.9968429184998953
2 fold: ls 0.003800983484315241 0.007470415894276413 auc 0.9993937340554103 0.9943706788183533
3 fold: ls 0.004138734992535417 0.006512420880988574 auc 0.9992476829293646 0.9919149537997359
4 fold: ls 0.003389169733367736 0.00688753627567146 auc 0.9995937655110224 0.9959902151821819
5 fold: ls 0.003983060077070462 0.006978652700091342 auc 0.999263575636618 0.9959312862740168
6 fold: ls 0.003486393333997919 0.006222674983988498 auc 0.999545413350069 0.995166519999162
7 fold: ls 0.004049546513380976 0.007573689534726002 auc 0.9992079308588462 0.9957230707985
8 fold: ls 0.003963451177583543 0.007162302425739552 auc 0.9992941440008463 0.9939536432609402
9 fold: ls 0.004355945431126567 0.007603329627634998 auc 0.9990487229256584 0.9819097446514281
threat 0.095 0.4 4
this class avg train 0.003918641778843698 avg val 0.0069402539594922135
this class auc train 0.9993161878844766 auc val 0.99385817867

2 fold: ls 0.04886998732871722 0.05536650805100901 auc 0.9921997859731232 0.9888806721789265
3 fold: ls 0.047109248296996996 0.051901707142502786 auc 0.9928260029909157 0.9905126000087
4 fold: ls 0.04670165460059791 0.05481334293433445 auc 0.9929561605732272 0.9892579141335873
5 fold: ls 0.04472935265535438 0.05228664806020755 auc 0.9936474100231216 0.990556063277597
6 fold: ls 0.04206670002549743 0.056356832352222545 auc 0.9945210673130226 0.9891096689648573
7 fold: ls 0.046964666554243806 0.055650454411322485 auc 0.9928636943227899 0.9894277124909417
8 fold: ls 0.048746503973445016 0.05245977685282157 auc 0.9922357452472966 0.9904578680370578
9 fold: ls 0.04827139923617239 0.0547917573083341 auc 0.9924165369052357 0.9892906711449142
insult 0.095 0.5 4
this class avg train 0.04717257199751616 avg val 0.05379163973751753
this class auc train 0.992788755291957 auc val 0.9898684633149291
0 fold: ls 0.013696294960621367 0.01731555032692072 auc 0.9964553239448667 0.9926880898862298
1 fold:

...

6 fold: ls 0.014400192466395224 0.016618228197026925 auc 0.9958101850502089 0.9905082556543103
7 fold: ls 0.015384518092343704 0.01868298475461082 auc 0.994877423152587 0.9879841390273865
8 fold: ls 0.014107041613256485 0.016798038465261827 auc 0.9960904217878608 0.992676493966327
9 fold: ls 0.015250027089136725 0.01679612185988656 auc 0.9951201805936767 0.9919236397138522
identity_hate 0.05 0.4 3
this class avg train 0.014798387258320545 avg val 0.017475299187045792
this class auc train 0.9954594273044671 auc val 0.9911510166547457
all loss avg 0.031940602716787746 0.035271680253339245
all auc avg 0.9945477532281062 0.991951959042355
FIND BETTER PARAMS 0.05 3 0.4 severe_toxic
FIND BETTER PARAMS 0.05 3 0.4 threat
FIND BETTER PARAMS 0.05 3 0.4 insult
TEST PARAM DONE ------------------------------------------
learning rate 0.05 max depth 3 feature fraction 0.5
0 fold: ls 0.0699966105979119 0.07793535371791159 auc 0.9906222750397026 0.9871180040263033
1 fold: ls 0.06870683736242793 0.0711

3 fold: ls 0.06983979218949386 0.07339745823793384 auc 0.9906588669020915 0.9889679834599027
4 fold: ls 0.06741270193086214 0.07812503238333891 auc 0.9914377720980595 0.9878862189881131
5 fold: ls 0.06807489243581936 0.07668913324607354 auc 0.9913034267747648 0.9875417104630684
6 fold: ls 0.0688250142078539 0.07813235678803032 auc 0.9910154667707481 0.9874609776100283
7 fold: ls 0.07193139366832126 0.07072960517919263 auc 0.9899497335563221 0.9901281492811761
8 fold: ls 0.06672564410504497 0.0739717223905832 auc 0.9916703648527927 0.989441759131684
9 fold: ls 0.06778541274279691 0.07780505430228057 auc 0.9912947133249533 0.9876472439696968
toxic 0.05 0.6 3
this class avg train 0.06891464907363011 avg val 0.07537885711738664
this class auc train 0.9909679477712745 auc val 0.9884645254291836
0 fold: ls 0.017381943163945143 0.02128573297742876 auc 0.9947101839216806 0.9910400050639321
1 fold: ls 0.017859798697693663 0.020422923519407533 auc 0.9942941230414267 0.9918391568552982
2 fold: ls

5 fold: ls 0.016036806926480903 0.01990583071404403 auc 0.9959163110732041 0.9923161199451248
6 fold: ls 0.016406304032856905 0.01920265397853668 auc 0.99565220083021 0.9928857600937684
7 fold: ls 0.014972232462173914 0.020161772078546098 auc 0.9967697796537657 0.9921026323364479
8 fold: ls 0.015916623687241376 0.021188474891964428 auc 0.9959991192396433 0.9910913743275034
9 fold: ls 0.01676455312752137 0.019114430297518553 auc 0.995401408648414 0.9930677069087633
severe_toxic 0.05 0.4 4
this class avg train 0.016202740253106855 avg val 0.020362482053407637
this class auc train 0.9958002011930605 auc val 0.9919124373457159
0 fold: ls 0.03286748363818009 0.037084155568726594 auc 0.9967921561523362 0.995658583053032
1 fold: ls 0.03408370651813966 0.036977346407150946 auc 0.9965482019208672 0.9954867023452907
2 fold: ls 0.032905865731366835 0.03889888872445383 auc 0.9967753351598176 0.9953962680232176
3 fold: ls 0.03267631490363722 0.03378124228666281 auc 0.9968292250457649 0.996475390065

6 fold: ls 0.03378668535071335 0.03737461446181567 auc 0.9965997748409516 0.995399792006666
7 fold: ls 0.03306711260919803 0.03819874128612743 auc 0.9967448962795189 0.995552576266833
8 fold: ls 0.03293282398889468 0.03893084789625736 auc 0.996778744975779 0.9951533480975188
9 fold: ls 0.0331108989032273 0.03884458638177997 auc 0.9967411381216362 0.9952608987176947
obscene 0.05 0.5 4
this class avg train 0.033268474576553755 avg val 0.037983306454102965
this class auc train 0.9967073260407748 auc val 0.995441875268325
0 fold: ls 0.0037978303461497567 0.006642016118303164 auc 0.9993994025209892 0.9969516027655563
1 fold: ls 0.004033613942123863 0.006245028596867936 auc 0.9992937725327597 0.9968141106222502
2 fold: ls 0.003974930035419054 0.007328402351658706 auc 0.9993007647231084 0.9943287764508695
3 fold: ls 0.004126481753537429 0.006546484801636763 auc 0.9992737674340644 0.9906100058667001
4 fold: ls 0.004103380829985995 0.006922180585549802 auc 0.9992539035802362 0.9961067634672198


7 fold: ls 0.004046390228723681 0.007587375182373839 auc 0.9992289235252996 0.9955803318876107
8 fold: ls 0.004159919337037268 0.007149154362142969 auc 0.9991459968825667 0.994068659115742
9 fold: ls 0.004185500834274946 0.007513218775244712 auc 0.9991513442764995 0.981985307393246
threat 0.05 0.6 4
this class avg train 0.003974570221761514 avg val 0.006919790729768886
this class auc train 0.9993010109543278 auc val 0.9939181576295384
0 fold: ls 0.04524948614729985 0.04985945781407136 auc 0.9935072520136924 0.9915208014749923
1 fold: ls 0.047340281582402284 0.05399877646745867 auc 0.9927434353516623 0.989749672911738
2 fold: ls 0.04920656692774303 0.0552006652234015 auc 0.9921132304999343 0.9890238464910374
3 fold: ls 0.04588021257164593 0.05168104062184838 auc 0.9932796101892793 0.9906169169045237
4 fold: ls 0.0485989554701862 0.05498693575021149 auc 0.9923049369244654 0.9891843771678346
5 fold: ls 0.04652055906626567 0.05261761278320113 auc 0.9930488084515422 0.9903987828502758
6 fol

In [5]:
def special_ens(model_name,k=3,rnd=233):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_auc_l,all_val_auc_l = 0,0
    
    params_list = [
        [0.05,4,0.5],
        [0.075,3,0.5],
        [0.075,3,0.6],
        [0.095,3,0.5],
        [0.075,4,0.5],
        [0.05,3,0.5],
    ]
    
    for i in range(6):
        val_loss_l,train_loss_l = 0,0
        val_auc_l,train_auc_l = 0,0
        fold_cnt = 0
        
        # special params
        params = {
                'application': 'binary',
                #'num_leaves': 8,
                #'lambda_l1': 1,
                'lambda_l2': 1.0,
                'max_depth': params_list[i][1],
                'metric': 'binary_logloss', # or auc
                'data_random_seed': 2,
                'learning_rate':params_list[i][0],
                'feature_fraction': params_list[i][2],

                }
        print(params)
            
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test = test_x
            s_round = 30
            # train for each class
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            try:
                train_pred = model.predict(curr_x)
                tmp_test_pred = model.predict(hold_out_x)
                
                curr_train_loss = log_loss(curr_y[:,i],train_pred)
                curr_val_loss = log_loss(hold_out_y[:,i],tmp_test_pred)
                
                curr_train_auc = roc_auc_score(curr_y[:,i],train_pred)
                curr_val_auc = roc_auc_score(hold_out_y[:,i],tmp_test_pred)
                
                print('ls',curr_train_loss,curr_val_loss,'auc',curr_train_auc,curr_val_auc)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
                val_auc_l += curr_val_auc
                train_auc_l += curr_train_auc
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_auc_l = train_auc_l/k
        val_auc_l = val_auc_l/k
        print(list_classes[i])
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class auc train',train_auc_l,'auc val',val_auc_l)
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/6
        all_val_loss_l += val_loss_l/6
        all_train_auc_l += train_auc_l/6
        all_val_auc_l += val_auc_l/6
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all auc avg',all_train_auc_l,all_val_auc_l)
    print('=======================================================')
    return test_pred

print('done')

lgb_res = special_ens('lgb',10,666)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = lgb_res
sample_submission.to_csv("../results/lgb_log_csv_fold10_stratified_special.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# best params changed when base models changed
# all loss avg 0.03111512578966158 0.03534320053008906 all auc avg 0.9948524214184179 0.9918450388634261

done
{'application': 'binary', 'max_depth': 4, 'feature_fraction': 0.5, 'metric': 'binary_logloss', 'lambda_l2': 1.0, 'data_random_seed': 2, 'learning_rate': 0.05}
0 fold: ls 0.067264566402172 0.07774020899970588 auc 0.9916054949158537 0.987196464391135
1 fold: ls 0.06580705500580764 0.07138156198895095 auc 0.9920510116182186 0.9898353510150018
2 fold: ls 0.06850507679428305 0.07616401241238943 auc 0.9911718996568857 0.9882864836166422
3 fold: ls 0.06625393858844372 0.07341956114133712 auc 0.9918871014931974 0.9889863301387463
4 fold: ls 0.06440112525776383 0.07772024618725268 auc 0.9924275736109777 0.9879902968267318
5 fold: ls 0.06824277558037303 0.07675832242927207 auc 0.9913147545822014 0.9873848230939658
6 fold: ls 0.06493381779223659 0.07796881259891172 auc 0.99229972609751 0.9875248476773688
7 fold: ls 0.06714251417653379 0.07084740200338364 auc 0.9916255379191474 0.9900778747500496
8 fold: ls 0.05622430850044298 0.07364552087798697 auc 0.9947001677450342 0.9894573537563076
9 fo